## Deliverable 2. Create a Customer Travel Destinations Map.

In [6]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [7]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
0,0,Fuling,29.7022,107.3919,84.70,72,100,2.71,CN,overcast clouds
1,1,Upernavik,72.7868,-56.1549,42.89,78,100,6.76,GL,overcast clouds
2,2,Bluff,-46.6000,168.3333,43.68,61,3,17.18,NZ,clear sky
3,3,Ushuaia,-54.8000,-68.3000,38.86,56,75,6.91,AR,broken clouds
4,4,Fortuna,40.5982,-124.1573,75.92,64,20,10.36,US,few clouds


In [8]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 

min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

In [9]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
new_city_data_df=city_data_df.loc[(city_data_df["Max Temp"]>=min_temp) & (city_data_df["Max Temp"]<=max_temp)]
new_city_data_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
0,0,Fuling,29.7022,107.3919,84.70,72,100,2.71,CN,overcast clouds
4,4,Fortuna,40.5982,-124.1573,75.92,64,20,10.36,US,few clouds
11,11,Monte Alegre,-2.0078,-54.0692,79.00,67,87,3.76,BR,overcast clouds
12,12,Pishin,30.5803,66.9961,77.94,59,55,4.56,PK,broken clouds
13,13,Bathsheba,13.2167,-59.5167,81.21,89,20,14.97,BB,few clouds


In [10]:
# 4a. Determine if there are any empty rows.
new_city_data_df.count()


Unnamed: 0             259
City                   259
Lat                    259
Lng                    259
Max Temp               259
Humidity               259
Cloudiness             259
Wind Speed             259
Country                259
Current Description    259
dtype: int64

In [11]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = new_city_data_df.dropna()

In [12]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Fuling,CN,84.70,overcast clouds,29.7022,107.3919,
4,Fortuna,US,75.92,few clouds,40.5982,-124.1573,
11,Monte Alegre,BR,79.00,overcast clouds,-2.0078,-54.0692,
12,Pishin,PK,77.94,broken clouds,30.5803,66.9961,
13,Bathsheba,BB,81.21,few clouds,13.2167,-59.5167,
14,Albany,US,85.93,scattered clouds,42.6001,-73.9662,
16,Balkanabat,TM,89.17,broken clouds,39.5108,54.3671,
17,Uribia,CO,85.86,clear sky,11.7139,-72.2660,
18,Biltine,TD,80.78,overcast clouds,14.5333,20.9167,
19,Labuhan,ID,78.75,scattered clouds,-6.8844,112.2051,


In [13]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
        
    except (IndexError):
        print("Hotel not found... skipping.")



Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [14]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head(20)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Fuling,CN,84.70,overcast clouds,29.7022,107.3919,Yingbin Hostel
4,Fortuna,US,75.92,few clouds,40.5982,-124.1573,Super 8 by Wyndham Fortuna
11,Monte Alegre,BR,79.00,overcast clouds,-2.0078,-54.0692,Hotel Shekinah
12,Pishin,PK,77.94,broken clouds,30.5803,66.9961,City Hotel Pishin
13,Bathsheba,BB,81.21,few clouds,13.2167,-59.5167,Atlantis Hotel
14,Albany,US,85.93,scattered clouds,42.6001,-73.9662,
16,Balkanabat,TM,89.17,broken clouds,39.5108,54.3671,Nebitchi Hotel
17,Uribia,CO,85.86,clear sky,11.7139,-72.2660,Hotel Uribia
18,Biltine,TD,80.78,overcast clouds,14.5333,20.9167,Hôtel Trois Étoiles de Biltine
19,Labuhan,ID,78.75,scattered clouds,-6.8844,112.2051,DoubleFAG


In [16]:
# 8a. Create the output File (CSV)
output_data_file = ("Vacation_Search/WeatherPy_vacation.csv")
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [17]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>City Name</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Weather Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [19]:
# 11a. Add a marker layer for each city to the map. 

locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)


# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))